In [2]:
import pandas as pd 
df = pd.read_csv("AWCustomers.csv")
df.head()


,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,LastUpdated
0,21173,NaN,Chad,C,Yuan,NaN,7090 C. Mount Hood,NaN,Wollongong,New South Wales,...,Bachelors,Clerical,M,M,1,3,0,1,81916,2017-03-06
1,13249,NaN,Ryan,NaN,Perry,NaN,3651 Willow Lake Rd,NaN,Shawnee,British Columbia,...,Partial College,Clerical,M,M,1,2,1,2,81076,2017-03-06
2,29350,NaN,Julia,NaN,Thompson,NaN,1774 Tice Valley Blvd.,NaN,West Covina,California,...,Bachelors,Clerical,F,S,0,3,0,0,86387,2017-03-06
3,13503,NaN,Theodore,NaN,Gomez,NaN,2103 Baldwin Dr,NaN,Liverpool,England,...,Partial College,Skilled Manual,M,M,1,2,1,2,61481,2017-03-06
4,22803,NaN,Marshall,J,Shan,NaN,Am Gallberg 234,NaN,Werne,Nordrhein-Westfalen,...,Partial College,Skilled Manual,M,S,1,1,0,0,51804,2017-03-06


In [3]:
df1 = df.drop(columns=['CustomerID', 'FirstName', 'MiddleName', 'LastName', 'Title', 'Suffix', 'AddressLine1', 'AddressLine2', 'PhoneNumber', 'PostalCode', 'City', 'StateProvinceName',  'TotalChildren', 'LastUpdated'])

#BirthDate to Age
from datetime import datetime
df1['BirthDate'] = pd.to_datetime(df1['BirthDate'])
today = pd.to_datetime('today')
df1['Age'] = df1['BirthDate'].apply(lambda x: today.year - x.year - ((today.month, today.day) < (x.month, x.day)))

df1 = df1.drop(columns=['BirthDate'])
df1.head()


,CountryRegionName,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,YearlyIncome,Age
0,Australia,Bachelors,Clerical,M,M,1,3,0,81916,37
1,Canada,Partial College,Clerical,M,M,1,2,1,81076,53
2,United States,Bachelors,Clerical,F,S,0,3,0,86387,39
3,United Kingdom,Partial College,Skilled Manual,M,M,1,2,1,61481,47
4,Germany,Partial College,Skilled Manual,M,S,1,1,0,51804,50


In [4]:
df1.isnull().sum()

CountryRegionName       0
Education               0
Occupation              0
Gender                  0
MaritalStatus           0
HomeOwnerFlag           0
NumberCarsOwned         0
NumberChildrenAtHome    0
YearlyIncome            0
Age                     0
dtype: int64

In [5]:
import numpy as np
from scipy.stats import zscore
z_scores = np.abs(zscore(df1[['NumberCarsOwned', 'NumberChildrenAtHome',
       'YearlyIncome']]))
threshold = 3

df1_new = df1[(z_scores < threshold).all(axis=1)]
print("Original shape:", df1.shape)
print("After removing outliers:", df1_new.shape)

Original shape: (18361, 10)
After removing outliers: (18327, 10)


In [6]:
df1_new['YearlyIncome'] = pd.qcut(df1_new['YearlyIncome'], q = 5, labels=['Low', 'Mid-Low','Mid','Mid-High','High'])
df1_new['Age'] = pd.cut(df1_new['Age'], bins=[0,30,60,df1_new['Age'].max()], labels=['Young', 'Mid-Aged','Old'])

df1_new.head()

/var/folders/4m/9q7vjyw12gxcw4cw7hmxp6gm0000gn/T/ipykernel_12017/1261897309.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_new['YearlyIncome'] = pd.qcut(df1_new['YearlyIncome'], q = 5, labels=['Low', 'Mid-Low','Mid','Mid-High','High'])
/var/folders/4m/9q7vjyw12gxcw4cw7hmxp6gm0000gn/T/ipykernel_12017/1261897309.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_new['Age'] = pd.cut(df1_new['Age'], bins=[0,30,60,df1_new['Age'].max()], labels=['Young', 'Mid-Aged','Old'])


,CountryRegionName,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,YearlyIncome,Age
0,Australia,Bachelors,Clerical,M,M,1,3,0,Mid-High,Mid-Aged
1,Canada,Partial College,Clerical,M,M,1,2,1,Mid-High,Mid-Aged
2,United States,Bachelors,Clerical,F,S,0,3,0,Mid-High,Mid-Aged
3,United Kingdom,Partial College,Skilled Manual,M,M,1,2,1,Mid,Mid-Aged
4,Germany,Partial College,Skilled Manual,M,S,1,1,0,Mid-Low,Mid-Aged


In [7]:
	
for col in df1_new.columns:
    print(df1_new[col].unique())

['Australia' 'Canada' 'United States' 'United Kingdom' 'Germany' 'France']
['Bachelors' 'Partial College' 'High School' 'Partial High School'
 'Graduate Degree']
['Clerical' 'Skilled Manual' 'Manual' 'Management' 'Professional']
['M' 'F']
['M' 'S']
[1 0]
[3 2 1 0 4]
[0 1 2]
['Mid-High', 'Mid', 'Mid-Low', 'Low', 'High']
Categories (5, object): ['Low' < 'Mid-Low' < 'Mid' < 'Mid-High' < 'High']
['Mid-Aged', 'Young', 'Old']
Categories (3, object): ['Young' < 'Mid-Aged' < 'Old']


In [8]:
from sklearn.preprocessing import OneHotEncoder
# pd.set_option('display.max_columns', None)

ohe = OneHotEncoder()
arr = ohe.fit_transform(df1_new).toarray()
data = pd.DataFrame(arr, columns=ohe.get_feature_names_out(['CountryRegionName','Education','Occupation','Gender','MaritalStatus','HomeOwnerFlag','NumberCarsOwned','NumberChildrenAtHome','YearlyIncome','Age']))
data = data.drop(columns=['HomeOwnerFlag_0'])
data.shape
data.head()

,CountryRegionName_Australia,CountryRegionName_Canada,CountryRegionName_France,CountryRegionName_Germany,CountryRegionName_United Kingdom,CountryRegionName_United States,Education_Bachelors,Education_Graduate Degree,Education_High School,Education_Partial College,...,NumberChildrenAtHome_1,NumberChildrenAtHome_2,YearlyIncome_High,YearlyIncome_Low,YearlyIncome_Mid,YearlyIncome_Mid-High,YearlyIncome_Mid-Low,Age_Mid-Aged,Age_Old,Age_Young
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [9]:
from sklearn.metrics.pairwise import cosine_similarity
obj1 = data.iloc[0]
obj2 = data.iloc[1]
# Simple Matching
matches = np.sum(obj1 == obj2)
smc = matches / len(obj1)
print(smc)

# Jaccard Similarity
intersection = np.sum(np.logical_and(obj1, obj2))
union = np.sum(np.logical_or(obj1, obj2))
jaccard = intersection / union
print(jaccard)

# Cosine Similarity
cosine = cosine_similarity([obj1], [obj2])[0][0]
print(cosine)

0.7837837837837838
0.42857142857142855
0.6


In [10]:
df1['Age'].corr(df1['YearlyIncome'])

np.float64(0.026512358727349694)